In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [10]:
from ml.models.base.slug_ann import SlugANN
from utils import helper, config, dasker
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [11]:
config.create_project_dirs(overwrite=True)

Cleared the existing saved models
Directory structure for project /covid created successfully


In [12]:
cl = dasker.get_local_client()
# cl =  dasker.get_global_client()
print(cl)

<Client: 'tcp://127.0.0.1:8786' processes=4 threads=4, memory=15.37 GiB>


In [13]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=config.rand_state)

In [14]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)
X_scalar = StandardScaler().fit_transform(X.copy())

X_base, X_test, y_base, y_test = train_test_split(X_scalar, y, test_size=0.33, random_state=config.rand_state)
X_train, X_val, y_train, y_val = train_test_split(X_base, y_base)


In [15]:
slug_ann = SlugANN()
slug_ann.X_train = X_train
slug_ann.X_test = X_test
slug_ann.X_val = X_val
slug_ann.y_train = y_train
slug_ann.y_test = y_test
slug_ann.y_val = y_val

slug_ann.n_trials = 1000    #### increase this number to imporve the goodness of fit


In [16]:
best_ann = slug_ann.fetch_model(retrain=True)

2022-10-25 19:27:53,849 - console_info - INFO - slug ann: Starting train for trials:1000 with epochs: 300
2022-10-25 19:27:53,849 - console_info - INFO - slug ann: Starting train for trials:1000 with epochs: 300
2022-10-25 19:27:53,856 - console_info - INFO - slug ann: Cleared previous models in the model save path
2022-10-25 19:27:53,856 - console_info - INFO - slug ann: Cleared previous models in the model save path
2022-10-25 20:10:05,013 - console_info - INFO - slug xgboost: Number of trials: 1000
2022-10-25 20:10:05,013 - console_info - INFO - slug xgboost: Number of trials: 1000
2022-10-25 20:10:05,157 - console_info - INFO - Best trial: 622
2022-10-25 20:10:05,157 - console_info - INFO - Best trial: 622
2022-10-25 20:10:05,428 - console_info - INFO - slug ann: Model saved at /home/wasif/python-asd/xai/app/ml/models/saved/covid/base/ann/slug/slug_ann_622.pickle
2022-10-25 20:10:05,428 - console_info - INFO - slug ann: Model saved at /home/wasif/python-asd/xai/app/ml/models/saved/

In [18]:
slug_ann.get_model_score(score_func=r2_score)

[0.5709415152455228, -0.2933985975774258]

In [19]:
pred = slug_ann.predict(X)

In [20]:
r2_score(pred, y)
#mean_squared_error(pred, y)

-0.07337026508459221